In [1]:
import sys
import py_compile
import pandas as pd
import xgboost as xgb
import numpy as np

sys.path.append(r"../")
py_compile.compile("../utility.py")
from utility import rmse
from sklearn.model_selection import train_test_split 

In [2]:
exclude = ['PC', 'SC-NA', 'SST-Indian', 'AO', 'NAO', 'PNA', 'nina3.4', 'nina1+2', 'nina3', 'nina4']

df = pd.read_csv("../feature-prepare/input-PC3.csv")

features = [feature for feature in list(df.keys()) if feature not in exclude]
target = ['PC']

predPC = []
obsPC = []

idx = [line for line in range(0,32)]
idxx = [line for line in range(32,42)]
train_df = df.iloc[idx][features]
train_target = df.iloc[idx][target]
test_df = df.iloc[idxx][features]
test_target = df.iloc[idxx][target]
    
param = {'nthread':6, #when use hyperthread, xgboost may become slower
              'learning_rate': 0.01, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 1, #5
              'silent': True,
              'subsample': 0.3,
              'colsample_bytree': 1.0,
              'n_estimators': 3000}

dtrain = xgb.DMatrix(train_df, label=train_target, feature_names=features)
dtest  = xgb.DMatrix(test_df, label=test_target, feature_names=features)
    
    
    
cv_res= xgb.cv(param,dtrain,num_boost_round=6000,early_stopping_rounds=3000,nfold=4, metrics='rmse',show_stdv=False)

bst = xgb.train(param,dtrain,num_boost_round=cv_res.shape[0])
pred = bst.predict(dtest)
    
print(pred)

oo = pd.DataFrame(pred)
oo.to_csv('PC3.txt',header=0,index=0)

[20:55:55] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:55:55] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:55:55] WARNING: ../src/learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you 

In [3]:
for impt_type in 'weight', 'gain', 'cover', 'total_gain', 'total_cover':
    print(impt_type + ":        " + str(bst.get_score(importance_type=impt_type)))

weight:        {'SST-Pacific': 72, 'SIC': 341, 'SST-Atlantic': 79, 'SC-TP': 139, 'Z70': 69, 'SC-EU': 141}
gain:        {'SST-Pacific': 610538.185873611, 'SIC': 360234.671931076, 'SST-Atlantic': 558872.7393034177, 'SC-TP': 763666.7283056835, 'Z70': 759961.8066123191, 'SC-EU': 418234.97173283674}
cover:        {'SST-Pacific': 6.805555555555555, 'SIC': 4.947214076246334, 'SST-Atlantic': 7.177215189873418, 'SC-TP': 7.273381294964029, 'Z70': 8.289855072463768, 'SC-EU': 5.815602836879433}
total_gain:        {'SST-Pacific': 43958749.38289999, 'SIC': 122840023.12849692, 'SST-Atlantic': 44150946.404970005, 'SC-TP': 106149675.23449, 'Z70': 52437364.656250015, 'SC-EU': 58971131.01432998}
total_cover:        {'SST-Pacific': 490.0, 'SIC': 1687.0, 'SST-Atlantic': 567.0, 'SC-TP': 1011.0, 'Z70': 572.0, 'SC-EU': 820.0}
